In [ ]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/kg192092/EP_JKS.git

GitHub user··········
GitHub password··········
Cloning into 'EP_JKS'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5205 (delta 0), reused 4 (delta 0), pack-reused 5200
Receiving objects: 100% (5205/5205), 601.45 MiB | 14.78 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (6173/6173), done.


In [ ]:
import tensorflow as tf

print(tf.version.VERSION)

import numpy as np
np.version.version

1.15.0


'1.17.5'

In [ ]:

import shutil
shutil.unpack_archive("/content/video.zip", "/content")

In [ ]:
model_dir = '/content/drive/My Drive/Embedded_project/Training'
!ls -ltra '{model_dir}'

In [ ]:
!pip install tf_slim

In [ ]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/kg192092/EP_JKS.git'

# Number of training steps.
num_steps = 1000  # 200000
#num_steps = 100  # 200000

# Number of evaluation steps.
num_eval_steps = 50


# model configs are from Model Zoo github: 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models
MODELS_CONFIG = {
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
    'ssd_mobilenet_v1_quantized': {
        'model_name': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18',
        'pipeline_file': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config',
        'batch_size': 12
    },    
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 12
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
# Note: for Edge TPU, you have to:
# 1) start with a pretrained model from model zoo, such as above 4
# 2) Must be a quantized model, which reduces the model size significantly
selected_model = 'ssd_mobilenet_v2_quantized'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
import os

%cd /content

repo_dir_path = "/content/EP_JKS/"

#!git clone {repo_url}
%cd {repo_dir_path}

print('Pull it so that we have the latest code/data')
!git pull

/content
/content/EP_JKS
Pull it so that we have the latest code/data
Already up to date.


In [ ]:
pip uninstall tensorflow

In [ ]:
!pip install tensorflow==1.15

In [ ]:
pip install --upgrade numpy==1.17.5

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
%cd /dataset/train/
# Generate `train.record`
!python /content/EP_JKS/code/generate_tfrecord.py --csv_input=/content/dataset/train/train.csv --output_path=/content/drive/My\ Drive/Embedded_project/Training/train.record --img_path=/content/dataset/train/images --label_map /content/drive/My\ Drive/Embedded_project/label_map.pbtxt



[Errno 2] No such file or directory: '/dataset/train/'
/content/models/research
Successfully created the TFRecords: /content/drive/My Drive/Embedded_project/Training/train.record


In [ ]:
# Generate `test.record`
!python /content/EP_JKS/code/generate_tfrecord.py --csv_input=/content/dataset/test/test.csv --output_path=/content/drive/My\ Drive/Embedded_project/Training/test.record --img_path=/content/dataset/test/images --label_map /content/drive/My\ Drive/Embedded_project/label_map.pbtxt

Successfully created the TFRecords: /content/drive/My Drive/Embedded_project/Training/test.record


In [ ]:
test_record_fname = "/content/drive/My\ Drive/Embedded_project/Training/test.record"
train_record_fname = "/content/drive/My\ Drive/Embedded_project/Training/train.record"
label_map_pbtxt_fname ="/content/label_map.pbtxt"

In [ ]:
%cd /content/dataset/test/
!cat test.csv

/content/dataset/test
filename,width,height,class,xmin,ymin,xmax,ymax
bottle_0.jpg,640,480,bottle,256,88,415,341
bottle_1.jpg,480,640,bottle,389,183,396,209
bottle_1.jpg,480,640,bottle,374,189,380,209
bottle_1.jpg,480,640,bottle,367,184,373,211
bottle_1.jpg,480,640,bottle,383,187,389,210
bottle_1.jpg,480,640,bottle,433,280,451,310
bottle_1.jpg,480,640,bottle,463,282,472,318
bottle_1.jpg,480,640,bottle,399,161,408,177
bottle_1.jpg,480,640,bottle,373,135,383,149
bottle_1.jpg,480,640,bottle,404,132,413,147
bottle_1.jpg,480,640,bottle,415,131,424,147
bottle_2.jpg,640,480,bottle,36,553,64,639
bottle_3.jpg,640,428,bottle,63,42,144,353
bottle_4.jpg,427,640,bottle,623,233,638,270
bottle_4.jpg,427,640,bottle,498,278,520,302
bottle_4.jpg,427,640,bottle,83,219,87,247
bottle_5.jpg,375,500,bottle,66,252,112,367
bottle_5.jpg,375,500,bottle,205,47,232,112
bottle_6.jpg,480,640,bottle,164,179,209,268
bottle_7.jpg,333,500,bottle,340,120,359,157
bottle_7.jpg,333,500,bottle,317,123,337,159
bottle_8.jpg,64

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 204M
drwx------  2 303230 5000 4.0K Jan  4  2019 .
drwxr-xr-x 63 root   root 4.0K Jun  7 08:16 ..
-rw-------  1 303230 5000  93M Jan  4  2019 model.ckpt.data-00000-of-00001
-rw-------  1 303230 5000  68K Jan  4  2019 model.ckpt.index
-rw-------  1 303230 5000  20M Jan  4  2019 model.ckpt.meta
-rw-------  1 303230 5000 4.3K Jan  4  2019 pipeline.config
-rw-------  1 303230 5000  24M Jan  4  2019 tflite_graph.pb
-rw-------  1 303230 5000  68M Jan  4  2019 tflite_graph.pbtxt


In [ ]:
fine_tune_checkpoint = "/content/drive/My Drive/Embedded_project/Training/model.ckpt-100000"
fine_tune_checkpoint

'/content/drive/My Drive/Embedded_project/Training/model.ckpt-100000'

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

# training pipeline file defines:
# - pretrain model path
# - the train/test sets
# - ID to Label mapping and number of classes
# - training batch size
# - epochs to trains
# - learning rate
# - etc

# note we just need to use a sample one, and make edits to it.

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint: downloaded pre-trained model checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test, we created earlier with our training/test sets
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path: ID to label file
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps (Number of epochs to train)
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
%cd /content/EP_JKS/dataset/labels
!cat /content/label_map.pbtxt

/content/EP_JKS/dataset/labels
item {
    id: 1
    name: 'bottle'
}

item {
    id: 2
    name: 'bin'
}

item {
    id: 3
    name: 'laptop'
}

In [ ]:
# look for num_classes: 6, since we have 5 different road signs and 1 person type (total of 6 types) 
!cat {pipeline_fname}

# Quantized trained SSD with Mobilenet v2 on MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspe

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-06-07 08:16:38--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.54.251.217, 54.85.41.146, 3.95.144.123, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.54.251.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.48MB/s    in 2.0s    

2020-06-07 08:16:41 (6.48 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
num_steps = 100000
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir='{model_dir}' \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

In [ ]:
!ls -ltra '{model_dir}'

total 1789219
-rw------- 1 root root 1141685397 Jun  6 15:18 train.record
-rw------- 1 root root   25719978 Jun  6 15:21 test.record
drwx------ 2 root root       4096 Jun  6 15:40 eval_0
-rw------- 1 root root   59305312 Jun  6 15:47 events.out.tfevents.1591457349.ff82d92a8549
drwx------ 3 root root       4096 Jun  6 15:47 fine_tuned_model
-rw------- 1 root root   59306427 Jun  6 15:52 events.out.tfevents.1591458591.ff82d92a8549
-rw------- 1 root root   31332964 Jun  6 17:12 graph.pbtxt
drwx------ 2 root root       4096 Jun  6 21:41 .ipynb_checkpoints
-rw------- 1 root root      68664 Jun  7 03:12 model.ckpt-94616.index
-rw------- 1 root root   74452352 Jun  7 03:12 model.ckpt-94616.data-00000-of-00001
-rw------- 1 root root   16444948 Jun  7 03:12 model.ckpt-94616.meta
-rw------- 1 root root      68664 Jun  7 03:22 model.ckpt-96076.index
-rw------- 1 root root   74452352 Jun  7 03:22 model.ckpt-96076.data-00000-of-00001
-rw------- 1 root root   16444948 Jun  7 03:22 model.ckpt-96076.m

In [ ]:
import os
import re
import numpy as np

output_directory = '%s/fine_tuned_model' % model_dir
os.makedirs(output_directory, exist_ok=True)

In [ ]:
lst = os.listdir(model_dir)
# find the last model checkpoint file, i.e. model.ckpt-1000.meta
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

/content/drive/My Drive/Embedded_project/Training/model.ckpt-100000


In [ ]:
!echo creates the frozen inference graph in fine_tune_model
# there is an "Incomplete shape" message.  but we can safely ignore that. 
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory='{output_directory}' \
    --trained_checkpoint_prefix='{last_model_path}'

creates the frozen inference graph in fine_tune_model
Instructions for updating:
Please use `layer.__call__` method instead.
W0607 08:17:15.560713 140515778631552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0607 08:17:17.456127 140515778631552 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0607 08:17:17.579927 140515778631552 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0607 08:17:17.613504 140515778631552 convolutional_box_predictor.py:156] depth of additional conv before box pred

In [ ]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={pipeline_fname} \
    --trained_checkpoint_prefix='{last_model_path}' \
    --output_directory='{output_directory}' \
    --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0607 08:18:01.628100 140365109655424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0607 08:18:03.608767 140365109655424 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0607 08:18:03.636020 140365109655424 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0607 08:18:03.662795 140365109655424 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [ ]:
!echo "CONVERTING frozen graph to quantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/custom_object_detection.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

CONVERTING frozen graph to quantized TF Lite file...
2020-06-07 08:18:14.983498: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-07 08:18:14.996134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-07 08:18:14.996705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-06-07 08:18:14.996950: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-06-07 08:18:14.998023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-06-07 08:18:14.999046: I tensorflow/stream_executor/platform/default/dso_l

In [ ]:
!echo "CONVERTING frozen graph to unquantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/Coo.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops 


CONVERTING frozen graph to unquantized TF Lite file...
2020-06-07 08:18:23.085408: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-07 08:18:23.097760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-07 08:18:23.098309: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-06-07 08:18:23.098536: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-06-07 08:18:23.099697: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-06-07 08:18:23.100735: I tensorflow/stream_executor/platform/default/dso

In [ ]:
print(output_directory)
!ls -ltra '{output_directory}'
#pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb") # this is main one
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")  # this is tflite graph
!cp '{label_map_pbtxt_fname}' '{output_directory}'

/content/drive/My Drive/Embedded_project/Training/fine_tuned_model
total 133541
-rw------- 1 root root  2159785 Jun  7 08:17 model.ckpt.meta
-rw------- 1 root root    23537 Jun  7 08:17 model.ckpt.index
-rw------- 1 root root 18726392 Jun  7 08:17 model.ckpt.data-00000-of-00001
-rw------- 1 root root       77 Jun  7 08:17 checkpoint
-rw------- 1 root root 19570272 Jun  7 08:17 frozen_inference_graph.pb
drwx------ 3 root root     4096 Jun  7 08:17 saved_model
-rw------- 1 root root     4192 Jun  7 08:17 pipeline.config
-rw------- 1 root root 19211873 Jun  7 08:18 tflite_graph.pb
-rw------- 1 root root 53702629 Jun  7 08:18 tflite_graph.pbtxt
-rw------- 1 root root  4743944 Jun  7 08:18 custom_object_detection.tflite
-rw------- 1 root root 18596280 Jun  7 08:18 Coo.tflite


In [ ]:
import os, os.path, shutil

folder_path = "/content/dataset/test/images"

images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for image in images:
    folder_name = image.split('.')[0]

    new_path = os.path.join(folder_path, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(folder_path, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname
print(PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  "/content/test_images/4L/4L"

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

/content/drive/My Drive/Embedded_project/Training/fine_tuned_model/frozen_inference_graph.pb
['/content/test_images/4L/4L/laptop_69.jpg', '/content/test_images/4L/4L/laptop_57.jpg', '/content/test_images/4L/4L/laptop_73.jpg', '/content/test_images/4L/4L/laptop_86.jpg', '/content/test_images/4L/4L/laptop_84.jpg', '/content/test_images/4L/4L/laptop_82.jpg', '/content/test_images/4L/4L/laptop_85.jpg', '/content/test_images/4L/4L/laptop_87.jpg', '/content/test_images/4L/4L/laptop_78.jpg', '/content/test_images/4L/4L/laptop_59.jpg', '/content/test_images/4L/4L/laptop_72.jpg', '/content/test_images/4L/4L/laptop_66.jpg', '/content/test_images/4L/4L/laptop_77.jpg', '/content/test_images/4L/4L/laptop_62.jpg', '/content/test_images/4L/4L/laptop_79.jpg', '/content/test_images/4L/4L/laptop_70.jpg', '/content/test_images/4L/4L/laptop_65.jpg', '/content/test_images/4L/4L/laptop_61.jpg', '/content/test_images/4L/4L/laptop_58.jpg', '/content/test_images/4L/4L/laptop_68.jpg', '/content/test_images/4L/4

In [ ]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict



/content/models/research/object_detection


In [ ]:
%matplotlib inline

print('Running inferences on %s' % TEST_IMAGE_PATHS)
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

In [ ]:
### Webcam Video Input

Use this command to run object detection with the model and video streaming from your computer's camera:

```
python3 detect_camera.py \
  --model models/mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite \
  --labels models/coco_labels.txt \
  --output images/processed.avi
```

The video output with detection plays on screen, and saves to `images/processed.avi`. Omit the `--output` argument to not save the video.

To compare the performance when not using the Edge TPU, try
running it again with the model that's *not* compiled for the Edge TPU:

```
python3 detect_camera.py \
  --model models/mobilenet_ssd_v2_coco_quant_postprocess.tflite \
  --labels models/coco_labels.txt \
  --output images/processed.avi
```

You will need to install open-cv to run this example:

```
pip3 install opencv-python
```
 117  python/examples/detection/detect_camera.py 
Viewed
@@ -0,0 +1,117 @@
"""Example using TF Lite to detect objects in a laptop camera."""


import argparse
import cv2
import time
import numpy as np

from PIL import Image
from PIL import ImageDraw

import detect
from detect_image import load_labels, make_interpreter, draw_objects
import tflite_runtime.interpreter as tflite

EDGETPU_SHARED_LIB = 'libedgetpu.so.1'

def draw_objects_and_info(draw, objs, labels, info):
  draw_objects(draw, objs, labels)
  draw.text((0,0), info, fill='green')

class MovingAvgPerf():
  def __init__(self, nticks=10):
    self.times = []
    self.nticks = nticks

  def tick(self, diff):
    self.times.append(diff)
    if len(self.times) > self.nticks:
      self.times.pop(0)

  def fps_str(self):
    fps = len(self.times) / sum(self.times)
    return '%.2f fps' % fps

class MovingWindowPerf(MovingAvgPerf):

  def tick(self):
    super().tick(time.monotonic())

  def fps_str(self):
    if len(self.times) == 1:
      fps = 0
    else:
      fps = len(self.times) / (self.times[-1] - self.times[0])
    return '%.2f fps' % fps


def main():
  parser = argparse.ArgumentParser(
      formatter_class=argparse.ArgumentDefaultsHelpFormatter)
  parser.add_argument('-m', '--model', required=True,
                      help='File path of .tflite file.')
  parser.add_argument('-l', '--labels',
                      help='File path of labels file.')
  parser.add_argument('-t', '--threshold', type=float, default=0.4,
                      help='Score threshold for detected objects.')
  parser.add_argument('-o', '--output',
                      help='File path for the result video with annotations (eg: images/processed.avi)')
  args = parser.parse_args()

  labels = load_labels(args.labels) if args.labels else {}
  interpreter = make_interpreter(args.model)
  interpreter.allocate_tensors()

  video_capture = cv2.VideoCapture(0)
  video_out = None

  perf = MovingWindowPerf()
  tpuperf = MovingAvgPerf()
  print('Running... Press q to quit.')
  while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Crop the central square of the video feed
    height, width, _ = frame.shape   # Get dimensions
    new_width = min(width, height)
    new_height = new_width

    left = int( (width - new_width)/2 )
    top = int( (height - new_height)/2 )
    right = int( (width + new_width)/2 )
    bottom = int( (height + new_height)/2 )

    image = Image.fromarray(frame[top:bottom,left:right])
    if video_out is None and args.output:
      video_out = cv2.VideoWriter(args.output, cv2.VideoWriter_fourcc('M','J','P','G'), 17, (new_width,new_height))

    # Inference
    scale = detect.set_input(interpreter, image.size,
                             lambda size: image.resize(size, Image.ANTIALIAS))
    start = time.monotonic()
    interpreter.invoke()
    tpuperf.tick(time.monotonic() - start)
    perf.tick()
    objs = detect.get_output(interpreter, args.threshold, scale)

    # Display the resulting frame
    msg = '%s\n%s tpu rate' % (perf.fps_str(), tpuperf.fps_str())
    draw_objects_and_info(ImageDraw.Draw(image), objs, labels, msg)
    cv2.imshow('Video', np.asarray(image))

    if video_out is not None:
      video_out.write(np.asarray(image))

    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

  # When everything is done, release the capture
  if video_out is not None:
    video_out.release()
  video_capture.release()
  cv2.destroyAllWindows()

if __na

In [ ]:
!git clone https://github.com/ilyakava/tflite.git

fatal: destination path 'tflite' already exists and is not an empty directory.


In [ ]:
cd /content/video

/content/video


In [ ]:
!pwd

/content/video


In [ ]:
!python detect_video.py --model custom_object_detection_edgetpu.tflite --labels /content/video/label_map.txt

[INFO] parsing class labels...
[INFO] loading Coral model...
Traceback (most recent call last):
  File "detect_video.py", line 38, in <module>
    model = DetectionEngine(args["model"])
  File "/usr/lib/python3/dist-packages/edgetpu/detection/engine.py", line 73, in __init__
    super().__init__(model_path)
  File "/usr/lib/python3/dist-packages/edgetpu/basic/basic_engine.py", line 92, in __init__
    self._engine = BasicEnginePythonWrapper.CreateFromFile(model_path)
RuntimeError: No Edge TPU device detected!


In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

!sudo apt-get update

!sudo apt-get install edgetpu-compiler

In [ ]:
!edgetpu_compiler /content/drive/My\ Drive/Embedded_project/Training/fine_tuned_model/custom_object_detection.tflite

In [ ]:
!sudo apt-get update

!sudo apt-get install python3-edgetpu

Hit:1 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Fetched 3,626 B in 1s (2,573 B/s)
Reading pac

In [ ]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf

def read_tensor_from_readed_frame(frame, input_height=224, input_width=224,
        input_mean=0, input_std=255):
  output_name = "normalized"
  float_caster = tf.cast(frame, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)
  return result

def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

def VideoSrcInit(paath):
    cap = cv2.VideoCapture(paath)
    flag, image = cap.read()
    if flag:
        print("Valid Video Path. Lets move to detection!")
    else:
        raise ValueError("Video Initialization Failed. Please make sure video path is valid.")
    return cap

def main():
  Labels_Path = "/content/drive/My Drive/Embedded_project/Training/label_map.txt"
  Model_Path = "/content/drive/My Drive/Embedded_project/Training/fine_tuned_model/custom_object_detection.tflite"
  input_path = "/content/WIN_20200607_11_42_36_Pro.mp4"

  ##Loading labels
  labels = load_labels(Labels_Path)

  ##Load tflite model and allocate tensors
  interpreter = tf.lite.Interpreter(model_path=Model_Path)
  interpreter.allocate_tensors()
  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  input_shape = input_details[0]['shape']

  ##Read video
  cap = VideoSrcInit(input_path)

  while True:
    ok, cv_image = cap.read()
    if not ok:
      break

    ##Converting the readed frame to RGB as opencv reads frame in BGR
    image = Image.fromarray(cv_image).convert('RGB')

    ##Converting image into tensor
    image_tensor = read_tensor_from_readed_frame(image ,224, 224)

    ##Test model
    interpreter.set_tensor(input_details[0]['index'], image_tensor)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    ## You need to check the output of the output_data variable and 
    ## map it on the frame in order to draw the bounding boxes.


    cv2.namedWindow("cv_image", cv2.WINDOW_NORMAL)
    cv2.imshow("cv_image",cv_image)

    ##Use p to pause the video and use q to termiate the program
    key = cv2.waitKey(10) & 0xFF
    if key == ord("q"):
      break
    elif key == ord("p"):
      cv2.waitKey(0)
      continue 
  cap.release()

if __name__ == '__main__':
  main()

Valid Video Path. Lets move to detection!


ValueError: ignored